In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox, interactive, Text

def kresling_crease(dia, pattern_height, degree, floors, n, x_offset=0, y_offset=0, export_pdf=False, file_name="kresling_pattern_A4.pdf"):
    """
    Draws a Kresling origami crease pattern and optionally exports to A4 PDF with 1 unit = 1 cm.
    x_offset and y_offset shift the pattern in cm.
    """
    pattern_width = 2 * (dia / 2) * np.pi
    b = pattern_width / n  # Horizontal unit
    floor_height = pattern_height / floors  # Vertical unit
    theta = np.deg2rad(90 - degree)
    dx = floor_height / np.tan(theta)  # Horizontal shift due to slant

    # A4 size in cm: 21 x 29.7
    fig_width_cm = 21
    fig_height_cm = 29.7

    # Set figure size in cm (matplotlib uses inches, so convert)
    fig, ax = plt.subplots(figsize=(fig_width_cm/2.54, fig_height_cm/2.54), dpi=300)

    # Store vertices
    vertices = []
    for floor in range(floors + 1):
        row_vertices = []
        for col in range(n + 1):
            x = col * b
            y = floor * floor_height
            # Apply horizontal shift for zigzag pattern
            if floor % 2 == 1:
                x += dx
            else:
                x -= dx
            # Apply user offset
            x += x_offset
            y += y_offset
            row_vertices.append((x, y))
        vertices.append(row_vertices)

    # Calculate red line length (between blue lines, i.e., between horizontal creases)
    red_line_length = math.sqrt(floor_height**2 + (2*dx)**2)

    # Output floor height, one side length, and one red line length
    print(f"Floor height: {floor_height:.4f} cm")
    print(f"One side length: {b:.4f} cm")
    print(f"Red line length between blue lines: {red_line_length:.4f} cm")

    # Draw vertical red zigzag creases
    for col in range(n + 1):
        x_vals = [vertices[floor][col][0] for floor in range(floors + 1)]
        y_vals = [vertices[floor][col][1] for floor in range(floors + 1)]
        ax.plot(x_vals, y_vals, 'r')

    # Draw horizontal blue creases
    for floor in range(floors + 1):
        x_vals = [vertices[floor][col][0] for col in range(n + 1)]
        y_vals = [vertices[floor][col][1] for col in range(n + 1)]
        ax.plot(x_vals, y_vals, 'b')

    # Draw diagonal green creases (triangles)
    for floor in range(floors):
        for col in range(n):
            v1 = vertices[floor][col]
            v2 = vertices[floor + 1][col + 1]
            v3 = vertices[floor + 1][col]
            v4 = vertices[floor][col + 1]
            if floor % 2 == 1:
                ax.plot([v3[0], v4[0]], [v3[1], v4[1]], 'g--', alpha=0.8)  # diagonal \
            else:
                ax.plot([v1[0], v2[0]], [v1[1], v2[1]], 'g--', alpha=0.8)  # diagonal /

    ax.set_aspect('equal')
    # Hide axis ticks, labels, and spines for clean output
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel("")
    ax.set_ylabel("")
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Set axis limits to fit A4 and center the pattern
    ax.set_xlim(0, fig_width_cm)
    ax.set_ylim(0, fig_height_cm)
    ax.invert_yaxis()  # Optional: origin at top-left like paper

    plt.tight_layout()

    if export_pdf:
        plt.savefig(file_name, format="pdf", bbox_inches='tight')
        print(f"Exported to {file_name}")
        plt.close(fig)
    else:
        plt.show()

# Use interactive instead of interact for immediate recalculation on slider change
ui = interactive(
    kresling_crease,
    dia=FloatSlider(value=4, min=1, max=10, step=1, description='Diameter (cm)'),
    pattern_height=FloatSlider(value=20, min=5, max=40, step=1, description='Height (cm)'),
    degree=FloatSlider(value=29, min=5, max=45, step=0.1, description='Degree Red to Green'),
    floors=IntSlider(value=6, min=1, max=10, step=1, description='Floors'),
    n=IntSlider(value=6, min=1, max=20, step=1, description='Sides'),
    x_offset=FloatSlider(value=0, min=0, max=10, step=0.1, description='X Offset (cm)'),
    y_offset=FloatSlider(value=0, min=0, max=10, step=0.1, description='Y Offset (cm)'),
    export_pdf=Checkbox(value=False, description='Export to PDF'),
    file_name=Text(value='kresling_pattern_A4.pdf', description='File Name')
)
display(ui)

interactive(children=(FloatSlider(value=4.0, description='Diameter (cm)', max=10.0, min=1.0, step=1.0), FloatS…